In [5]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# model_name = 'bn22/Mistral-7B-Instruct-v0.1-sharded'
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [2]:
def load_quantized_model(model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

In [3]:
def initialize_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1
    return tokenizer

In [4]:
model = load_quantized_model(model_name)
tokenizer = initialize_tokenizer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [16]:
tokenizer.pad_token_id = tokenizer.eos_token_id

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
stop_token_ids = [0]

text = "[INST] How AI will replace Engineers [/INST]"

In [21]:
def generate_response(text):
    encoded = tokenizer(text, return_tensors="pt", add_special_tokens=True)
    encoded = encoded.to(device)
    generated_ids = model.generate(**encoded, max_new_tokens=512, do_sample=True, temperature=0.8)
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0][len(text):]

In [22]:
response = generate_response(text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [23]:
print(response)

NST] It is important to clarify that AI is a tool that can assist engineers in their work, rather than a replacement for them. Engineers bring a unique skill set and expertise that cannot be replicated by AI. Here are some ways that AI can augment the work of engineers, but cannot replace them entirely:

1. Design and analysis: AI can be used to analyze vast amounts of data and identify patterns that engineers may not be able to notice. This can help engineers to design more efficient systems, optimize processes, and make more informed decisions.
2. Predictive maintenance: AI can be used to monitor equipment and predict when maintenance is required. This can help engineers to prevent downtime and reduce costs.
3. Quality control: AI can be used to inspect products and identify defects. This can help engineers to ensure that products meet quality standards and are free from defects.
4. Hazardous environments: AI can be used to perform tasks in hazardous environments, such as space explo

In [24]:
text = """evaluate this ielts writing according to 4 criteria of ielts writing: \n
Space travel has been a reality for decades, and with the advances in technology, space tourism is believed to be full of potential. While there are both benefits and drawbacks to space tourism, I ultimately believe that it is a positive development.

On the one hand, certain negative aspects of space tourism justify the doubts surrounding it. Firstly, it could be exorbitant due to the cost of equipment and preparation involved, limiting its availability to a select group of extremely wealthy people. Space tourism would, as a result, likely become a status symbol to further drive a wedge between the rich and the poor in society. In addition, there are possible environmental concerns, as rockets used for space travel discharge a great amount of emission. As commercial flights have already been criticized for their impact on the environment, popularizing space tourism may deteriorate the situation by adding another source of air pollution.

Nevertheless, the possible advantages of space tourism are significant. Economically, it could open up a new industry, creating jobs and stimulating growth. This would be especially beneficial for the manufacturing and engineering sectors, providing a much-needed boost for countries with a competitive edge in said sectors. Space tourism, furthermore, is able to offer travelers breathtaking perspectives of the Earth, so they can appreciate the meaning of life on our planet. Many astronauts have recounted their experience of seeing the Earth from space as a transformative influence in that it helped them see past the wars and struggles on the planet. Consequently, ordinary people who embark on such a trip could reach the same conclusion, increasing the overall positivity among mankind. It can be argued from these points that the benefits of space tourism would overshadow its drawbacks, justifying the effort to help it progress.

In my opinion, developing space travel for recreation is largely positive, both economically and educationally. Because of the existing concerns, however, it is ultimately essential to weigh both the advantages and disadvantages before making the decision for it.
"""

In [25]:
res = generate_response(text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [26]:
print(res)

it.

1. Task response: The task requires the writer to express their opinion on the topic of space tourism. The writer clearly states their position at the beginning and throughout the essay, providing reasons and evidence to support their viewpoint. The writer also considers the counter-argument and acknowledges the potential drawbacks of space tourism.

2. Cohesion and coherence: The writer maintains a clear focus on the topic throughout the essay, providing reasons why they believe space tourism is a positive development. The writer uses logical progression to present their argument, starting with potential drawbacks before moving on to the advantages. The essay is also well-structured, with a clear introduction, body, and conclusion.

3. Vocabulary and grammar: The writer uses a range of vocabulary to discuss the topic, demonstrating a good command of English. The writer also uses accurate grammar throughout the essay, with no significant errors that would affect meaning.

4. Organ